<a href="https://colab.research.google.com/github/mhsnur/TESIS-S2-Geomatika-FT-UGM-Muhsin-Nur-Alamsyah/blob/main/BO_DBSCAN/Regularisasi_BO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##REGULARISASI BO

In [ ]:
!pip -q install geopandas shapely pyproj pyogrio rdp tqdm

  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import math, statistics
import numpy as np
import geopandas as gpd
from pyproj import CRS
from tqdm import tqdm
from rdp import rdp
from shapely import speedups
from shapely.geometry import Polygon, MultiPolygon, LineString, MultiLineString, shape
speedups.enable()

/tmp/ipython-input-3381388768.py:9: FutureWarning: This function has no longer any effect, and will be removed in a future release. Starting with Shapely 2.0, equivalent speedups are always available
  speedups.enable()


In [ ]:
# ================= RIGHT-ANGLE REGULARIZATION v2 (anti-stair & anti-teeth) =================
!pip -q install geopandas shapely pyproj rtree pyogrio

import math, numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.affinity import rotate as shp_rotate
from shapely.validation import make_valid
from shapely.ops import unary_union
from tqdm.auto import tqdm

# ---------- PATH ----------
IN_PATH  = "/content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline.geojson"
OUT_PATH = "/content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline_REGULARISASI.geojson"

# ---------- PARAMETER UTAMA ----------
# Klasifikasi & penyikuan
ANG_CLASSIFY_DEG   = 25.0   # deviasi <= nilai ini dari H/V dipaksa H/V
ANG_FORCE_DEG      = 45.0   # selain itu: paksa ke sumbu terdekat (H jika <45°, V jika >=45°)
MERGE_LINE_TOL     = 0.40   # m, merge posisi garis H/V yang berdekatancollinear_tol_deg = 2.0    # hapus vertex hampir kolinear
CLOSE_TOL          = 0.30   # m, close lekukan kecil (morphological closing)
SMOOTH_TOL         = 0.10   # m, smoothing mikro sebelum/sesudah penyikuan
GRID_SNAP          = 0.05   # m, pembulatan grid (0=off)
MAX_SNAP_OFFSET    = 1.80   # m, jaga agar pergeseran tidak kebablasan
COLLINEAR_TOL_DEG  = 2.0    # hapus vertex hampir kolinear


# Anti-stair & anti-teeth
TEETH_MAX_LEN      = 1.00   # m, dua segmen orthogonal pendek ≤ ini dianggap "gigi": dilompati
STAIR_SEG_MAX      = 1.50   # m, segmen ≤ ini dipertimbangkan sebagai anak-tangga
STAIR_MIN_PAIRS    = 2      # minimal pasangan HV untuk dianggap tangga (HVHV = 2 pasangan)
STAIR_MAX_DEPTH    = 0.60   # m, total deviasi tegak-lurus maksimum dari rangkaian tangga agar boleh diratakan

# Area-preserving (jangan mengecil)
AREA_TOL           = 1e-4   # toleransi rasio area
AREA_MATCH         = True   # area hasil ≥ area asal (buffer miter adaptif bila perlu)

# ---------- UTIL ----------
def to_work_crs(gdf):
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326", allow_override=True)
    crs = gdf.estimate_utm_crs() if gdf.crs.is_geographic else gdf.crs
    return gdf.to_crs(crs), crs

def dominant_angle_deg(geom):
    mrr = geom.minimum_rotated_rectangle
    if not isinstance(mrr, Polygon): return 0.0
    xs, ys = mrr.exterior.coords.xy
    pts   = np.array(list(zip(xs, ys)))
    edges = np.diff(pts, axis=0)[:4]
    lens  = np.hypot(edges[:,0], edges[:,1])
    if len(lens)==0 or np.max(lens)==0: return 0.0
    v   = edges[np.argmax(lens)]
    ang = math.degrees(math.atan2(v[1], v[0]))
    return (ang + 180) % 180

def _merge_vals(vals, tol):
    if not vals: return []
    vals = sorted(vals)
    groups = [[vals[0]]]
    for v in vals[1:]:
        if abs(v - groups[-1][-1]) <= tol: groups[-1].append(v)
        else: groups.append([v])
    return [float(np.mean(g)) for g in groups]

def _remove_colinear_ring(xy, tol_deg=2.0):
    pts = np.asarray(xy, float)
    if len(pts) < 4: return pts
    out = [pts[0]]
    for i in range(1, len(pts)-1):
        a = out[-1]; b = pts[i]; c = pts[i+1]
        v1 = b - a;  v2 = c - b
        if np.allclose(v1, 0) or np.allclose(v2, 0): continue
        ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
        if abs(180 - ang) <= tol_deg or ang <= tol_deg:  # hampir kolinear
            continue
        out.append(b)
    out.append(pts[-1])
    if (out[0] != out[-1]).any(): out.append(out[0])
    return np.asarray(out)

def _line_runs_classify(coords, ang_class=25, ang_force=45):
    pts = np.asarray(coords, float); H,V = 0,1; labs=[]
    for i in range(len(pts)-1):
        dx, dy = pts[i+1][0]-pts[i][0], pts[i+1][1]-pts[i][1]
        if abs(dx)<1e-12 and abs(dy)<1e-12:
            labs.append(H if (not labs or labs[-1]==H) else V); continue
        a = abs(math.degrees(math.atan2(dy, dx))); a=min(a, 180-a)
        if a <= ang_class: labs.append(H)
        elif abs(a-90) <= ang_class: labs.append(V)
        else: labs.append(H if a < ang_force else V)
    return labs

def _apply_runs_snap(coords, labels, merge_tol=0.3, grid_snap=0.0, max_shift=None):
    pts = np.asarray(coords, float)
    n = len(pts)-1; runs=[]; s=0
    for i in range(1, n):
        if labels[i] != labels[i-1]:
            runs.append((s, i-1, labels[i-1])); s = i
    runs.append((s, n-1, labels[-1]))

    xs, ys = [], []
    for a,b,lab in runs:
        seg = pts[a:b+2]
        if lab==0: ys.append(np.mean(seg[:,1]))   # H -> samakan Y
        else:      xs.append(np.mean(seg[:,0]))   # V -> samakan X
    xs = _merge_vals(xs, merge_tol); ys = _merge_vals(ys, merge_tol)

    out = pts.copy()
    for a,b,lab in runs:
        if lab==0:
            y0 = np.mean(out[a:b+2,1]);  y0 = min(ys, key=lambda yv: abs(yv-y0)) if ys else y0
            for k in range(a, b+2):
                dy = y0 - out[k,1]
                if (max_shift is None) or (abs(dy) <= max_shift): out[k,1] = y0
        else:
            x0 = np.mean(out[a:b+2,0]);  x0 = min(xs, key=lambda xv: abs(xv-x0)) if xs else x0
            for k in range(a, b+2):
                dx = x0 - out[k,0]
                if (max_shift is None) or (abs(dx) <= max_shift): out[k,0] = x0

    if grid_snap>0:
        out[:,0] = np.round(out[:,0]/grid_snap)*grid_snap
        out[:,1] = np.round(out[:,1]/grid_snap)*grid_snap

    return _remove_colinear_ring(out, COLLINEAR_TOL_DEG)

# ---- NEW: Hilangkan "gigi" HV/VH super pendek
def _flatten_teeth(coords, teeth_max=1.0):
    pts = np.asarray(coords, float).copy()
    def lab(a,b):
        return 0 if abs(b[1]-a[1]) < 1e-12 else 1  # setelah snap: H => y sama; V => x sama
    i=0
    while i < len(pts)-2:
        L0 = np.hypot(*(pts[i+1]-pts[i])); L1 = np.hypot(*(pts[i+2]-pts[i+1]))
        if L0<=teeth_max and L1<=teeth_max and lab(pts[i],pts[i+1]) != lab(pts[i+1],pts[i+2]):
            # hapus p[i+1] -> loncati ujung gigi
            pts = np.vstack([pts[:i+1], pts[i+2:]])
            if i>0: i-=1
            continue
        i+=1
    if (pts[0]!=pts[-1]).any(): pts=np.vstack([pts, pts[0]])
    return pts

# ---- NEW: Rata-kan "anak tangga" (zig-zag H/V pendek)
def _flatten_staircase(coords, seg_max=1.5, min_pairs=2, max_depth=0.6):
    pts = np.asarray(coords, float)
    if len(pts)<4: return pts
    def seg_lab_len(a,b):
        v=b-a; L=float(np.hypot(*v))
        lab = 0 if abs(v[1])<abs(v[0]) else 1  # kira2 H kalau |dx|>|dy| (setelah snap umumnya eksak)
        return lab, L
    out=[pts[0]]; i=0
    while i < len(pts)-1:
        j=i; labs=[]; lens=[]; xs=[pts[i][0]]; ys=[pts[i][1]]
        # kumpulkan HVHV.. dengan segmen pendek
        while j < len(pts)-1:
            lab,L = seg_lab_len(pts[j], pts[j+1])
            if L>seg_max: break
            if labs and lab==labs[-1]: break
            labs.append(lab); lens.append(L)
            j+=1; xs.append(pts[j][0]); ys.append(pts[j][1])
        pairs=len(labs)//2
        if pairs>=min_pairs:
            # cek kedalaman "zigzag"
            dx= max(xs)-min(xs); dy= max(ys)-min(ys)
            if min(dx,dy) <= max_depth:
                # ganti dengan garis lurus H atau V sesuai span dominan
                if dx >= dy:  # rata horizontal
                    y0=float(np.median(ys))
                    out.append(np.array([pts[j][0], y0]))
                else:         # rata vertikal
                    x0=float(np.median(xs))
                    out.append(np.array([x0, pts[j][1]]))
                i=j
                continue
        # bukan tangga -> keep titik berikutnya
        out.append(pts[i+1]); i+=1
    out=np.asarray(out)
    if (out[0]!=out[-1]).any(): out=np.vstack([out, out[0]])
    return _remove_colinear_ring(out, COLLINEAR_TOL_DEG)

def _make_ortho(poly, med_angle, close_t=CLOSE_TOL, smooth_t=SMOOTH_TOL):
    # smoothing/closing kecil agar ring stabil
    q = poly
    if smooth_t>0: q = q.buffer(smooth_t, join_style=2).buffer(-smooth_t, join_style=2)
    if close_t>0:  q = q.buffer(close_t,  join_style=2).buffer(-close_t,  join_style=2)
    q = make_valid(q).buffer(0)

    # putar ke sumbu XY
    rot = shp_rotate(q, -med_angle, origin='centroid', use_radians=False)

    def process_one(p):
        # EXTERIOR
        ext = np.asarray(p.exterior.coords)
        labs = _line_runs_classify(ext, ANG_CLASSIFY_DEG, ANG_FORCE_DEG)
        ext2 = _apply_runs_snap(ext, labs, MERGE_LINE_TOL, GRID_SNAP, MAX_SNAP_OFFSET)
        ext2 = _flatten_teeth(ext2, TEETH_MAX_LEN)
        ext2 = _flatten_staircase(ext2, STAIR_SEG_MAX, STAIR_MIN_PAIRS, STAIR_MAX_DEPTH)

        # HOLES
        holes=[]
        for r in p.interiors:
            arr  = np.asarray(r.coords)
            labs = _line_runs_classify(arr, ANG_CLASSIFY_DEG, ANG_FORCE_DEG)
            rr   = _apply_runs_snap(arr, labs, MERGE_LINE_TOL, GRID_SNAP, MAX_SNAP_OFFSET)
            rr   = _flatten_teeth(rr, TEETH_MAX_LEN)
            rr   = _flatten_staircase(rr, STAIR_SEG_MAX, STAIR_MIN_PAIRS, STAIR_MAX_DEPTH)
            if len(rr) >= 4: # Check if hole has at least 4 coordinates
              holes.append(rr)

        # Check if exterior ring has at least 4 coordinates before creating Polygon
        if len(ext2) >= 4:
            return Polygon(ext2, holes).buffer(0)
        else:
            return None # Return None if exterior ring is invalid


    if isinstance(rot, Polygon):
        out_rot = process_one(rot)
    else:
        parts = [process_one(pg) for pg in rot.geoms]
        parts = [pp for pp in parts if pp is not None and not pp.is_empty] # Filter out None and empty geometries
        out_rot = unary_union(parts).buffer(0)


    # putar kembali
    if out_rot is None or out_rot.is_empty: return out_rot
    out = shp_rotate(out_rot, med_angle, origin='centroid', use_radians=False)
    return make_valid(out).buffer(0)


def _area_match(poly, target, tol=AREA_TOL):
    A0 = poly.area
    if A0 >= target*(1.0 - tol) or not AREA_MATCH: return poly
    lo, hi = 0.0, max(0.05, (target - A0)/max(poly.length, 1e-9) * 2.0)
    def area_of(d): return poly.buffer(d, join_style=2, mitre_limit=5.0).area
    while area_of(hi) < target and hi < 5.0: hi *= 2.0
    for _ in range(30):
        mid = 0.5*(lo+hi); Am = area_of(mid)
        lo = mid if Am < target else lo; hi = hi if Am < target else mid
    return poly.buffer(hi, join_style=2, mitre_limit=5.0)

# ---------- BACA & PROYEKSI ----------
gdf = gpd.read_file(IN_PATH, engine="pyogrio")
gdf, work_crs = to_work_crs(gdf)

# bersihkan ringan
gdf["geometry"] = gdf.geometry.apply(lambda g: make_valid(g).buffer(0))
gdf = gdf[~gdf.geometry.is_empty].copy()

# ---------- PROSES PER-FITUR ----------
def regularize_right_angle_v2(geom):
    if geom is None or geom.is_empty: return geom
    theta = dominant_angle_deg(geom)              # ikut orientasi data input
    ortho = _make_ortho(geom, theta, CLOSE_TOL, SMOOTH_TOL)
    if ortho is None or ortho.is_empty: return geom # Handle None returned by _make_ortho
    ortho2 = _area_match(ortho, geom.area, AREA_TOL)  # jaga area tidak mengecil
    return ortho2.buffer(0)

out_geoms=[]
for g in tqdm(gdf.geometry, desc="Right-angle v2"):
    out_geoms.append(regularize_right_angle_v2(g))

gdf_out = gdf.copy()
gdf_out["geometry"] = out_geoms
gdf_out = gdf_out[~gdf_out.geometry.is_empty].copy()
gdf_out.to_file(OUT_PATH, driver="GeoJSON", engine="pyogrio")
print("Selesai →", OUT_PATH)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 13.3 MB/s eta 0:00:00


Right-angle v2:   0%|          | 0/54 [00:00<?, ?it/s]

/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipy

Selesai → /content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline_REGULARISASI.geojson


/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-2569095125.py:74: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipy

##REFINEMENT REGULARISASI

In [ ]:
# ================= RIGHT-ANGLE REGULARIZATION v3 — targeted cleanup =================
# Jalankan ini setelah hasil v2. Jika mau langsung dari GeoJSON awal juga bisa.

!pip -q install geopandas shapely pyproj rtree pyogrio

import math, numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing
from shapely.affinity import rotate as shp_rotate
from shapely.validation import make_valid
from shapely.ops import unary_union
from tqdm.auto import tqdm

# ---------- PATH ----------
IN_PATH  = "/content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline_REGULARISASI.geojson"  # hasil v2
OUT_PATH = "/content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline_FINISH.geojson"

# ---------- PARAMETER INTI (sesuaikan bila perlu) ----------
# 1) anti-stair/gerigi
STAIR_SEG_MAX   = 2.5   # m; segmen <= ini dianggap anak tangga
STAIR_MIN_PAIRS = 2     # HVHV (atau VH VH) minimal
STAIR_MAX_DEPTH = 0.8   # m; kedalaman zigzag maksimum agar diratakan

# 2) anti-spike / notch (ujung runcing)
SPIKE_ANGLE_MAX = 70.0  # derajat; sudut di vertex < ini dianggap tajam
SPIKE_DEPTH_MAX = 0.75  # m; jarak apex ke garis tetangga (base) <= ini → hapus apex

# 3) perbaikan sudut bevel (pojok miring pendek di antara H & V)
BEVEL_DIAG_MAX  = 2.0   # m; panjang diagonal pendek penghubung H↔V
RIGHT_TOL_DEG   = 15.0  # derajat; toleransi mendeteksi H/V pada data yang sudah diputar
CORNER_SHIFT_MAX= 1.8   # m; batas geser saat membentuk sudut 90°

# 4) utilitas ortho (menyelaraskan ke H/V lokal)
ANG_CLASSIFY_DEG = 25.0
ANG_FORCE_DEG    = 45.0
MERGE_LINE_TOL   = 0.40
GRID_SNAP        = 0.05
COLLINEAR_TOL_DEG= 2.0
CLOSE_TOL        = 0.30
SMOOTH_TOL       = 0.10

AREA_TOL   = 1e-4
AREA_MATCH = True

# ---------- UTIL ----------
def to_work_crs(gdf):
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326", allow_override=True)
    crs = gdf.estimate_utm_crs() if gdf.crs.is_geographic else gdf.crs
    return gdf.to_crs(crs), crs

def dominant_angle_deg(geom):
    mrr = geom.minimum_rotated_rectangle
    if not isinstance(mrr, Polygon): return 0.0
    xs, ys = mrr.exterior.coords.xy
    pts   = np.array(list(zip(xs, ys)))
    edges = np.diff(pts, axis=0)[:4]
    lens  = np.hypot(edges[:,0], edges[:,1])
    if len(lens)==0 or np.max(lens)==0: return 0.0
    v   = edges[np.argmax(lens)]
    ang = math.degrees(math.atan2(v[1], v[0]))
    return (ang + 180) % 180

def _merge_vals(vals, tol):
    if not vals: return []
    vals = sorted(vals)
    groups = [[vals[0]]]
    for v in vals[1:]:
        if abs(v - groups[-1][-1]) <= tol: groups[-1].append(v)
        else: groups.append([v])
    return [float(np.mean(g)) for g in groups]

def _remove_colinear_ring(xy, tol_deg=2.0):
    pts = np.asarray(xy, float)
    if len(pts) < 4: return pts
    out = [pts[0]]
    for i in range(1, len(pts)-1):
        a = out[-1]; b = pts[i]; c = pts[i+1]
        v1 = b - a;  v2 = c - b
        if np.allclose(v1, 0) or np.allclose(v2, 0): continue
        ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
        if abs(180 - ang) <= tol_deg or ang <= tol_deg:
            continue
        out.append(b)
    out.append(pts[-1])
    if (out[0] != out[-1]).any(): out.append(out[0])
    return np.asarray(out)

def _line_runs_classify(coords, ang_class=25, ang_force=45):
    pts = np.asarray(coords, float); H,V = 0,1; labs=[]
    for i in range(len(pts)-1):
        dx, dy = pts[i+1][0]-pts[i][0], pts[i+1][1]-pts[i][1]
        if abs(dx)<1e-12 and abs(dy)<1e-12:
            labs.append(H if (not labs or labs[-1]==H) else V); continue
        a = abs(math.degrees(math.atan2(dy, dx))); a=min(a, 180-a)
        if a <= ang_class: labs.append(H)
        elif abs(a-90) <= ang_class: labs.append(V)
        else: labs.append(H if a < ang_force else V)
    return labs

def _apply_runs_snap(coords, labels, merge_tol=0.3, grid_snap=0.0, max_shift=None):
    pts = np.asarray(coords, float)
    n = len(pts)-1; runs=[]; s=0
    for i in range(1, n):
        if labels[i] != labels[i-1]:
            runs.append((s, i-1, labels[i-1])); s = i
    runs.append((s, n-1, labels[-1]))

    xs, ys = [], []
    for a,b,lab in runs:
        seg = pts[a:b+2]
        if lab==0: ys.append(np.mean(seg[:,1]))
        else:      xs.append(np.mean(seg[:,0]))
    xs = _merge_vals(xs, merge_tol); ys = _merge_vals(ys, merge_tol)

    out = pts.copy()
    for a,b,lab in runs:
        if lab==0:
            y0 = np.mean(out[a:b+2,1]);  y0 = min(ys, key=lambda yv: abs(yv-y0)) if ys else y0
            for k in range(a, b+2):
                dy = y0 - out[k,1]
                if (max_shift is None) or (abs(dy) <= max_shift): out[k,1] = y0
        else:
            x0 = np.mean(out[a:b+2,0]);  x0 = min(xs, key=lambda xv: abs(xv-x0)) if xs else x0
            for k in range(a, b+2):
                dx = x0 - out[k,0]
                if (max_shift is None) or (abs(dx) <= max_shift): out[k,0] = x0

    if grid_snap>0:
        out[:,0] = np.round(out[:,0]/grid_snap)*grid_snap
        out[:,1] = np.round(out[:,1]/grid_snap)*grid_snap

    return _remove_colinear_ring(out, COLLINEAR_TOL_DEG)

# ---------- TARGETED CLEANERS ----------
# A. hapus "gigi" HV/VH superpendek (dua segmen kawin)
def _flatten_teeth(coords, teeth_max=1.2):
    pts = np.asarray(coords, float).copy()
    def lab(a,b):  # setelah snap: H => y sama; V => x sama
        return 0 if abs(b[1]-a[1]) < 1e-12 else 1
    i=0
    while i < len(pts)-2:
        L0 = float(np.hypot(*(pts[i+1]-pts[i])))
        L1 = float(np.hypot(*(pts[i+2]-pts[i+1])))
        if L0<=teeth_max and L1<=teeth_max and lab(pts[i],pts[i+1]) != lab(pts[i+1],pts[i+2]):
            pts = np.vstack([pts[:i+1], pts[i+2:]])  # loncati p[i+1]
            if i>0: i-=1
            continue
        i+=1
    if (pts[0]!=pts[-1]).any(): pts=np.vstack([pts, pts[0]])
    return pts

# B. ratakan rangkaian "anak tangga" pendek
def _flatten_staircase(coords, seg_max=2.5, min_pairs=2, max_depth=0.8):
    pts = np.asarray(coords, float)
    if len(pts)<4: return pts
    def seg_lab_len(a,b):
        v=b-a; L=float(np.hypot(*v))
        lab = 0 if abs(v[1])<abs(v[0]) else 1  # H bila |dx|>|dy|
        return lab, L
    out=[pts[0]]; i=0
    while i < len(pts)-1:
        j=i; labs=[]; xs=[pts[i][0]]; ys=[pts[i][1]]
        while j < len(pts)-1:
            lab,L = seg_lab_len(pts[j], pts[j+1])
            if L>seg_max: break
            if labs and lab==labs[-1]: break
            labs.append(lab); j+=1; xs.append(pts[j][0]); ys.append(pts[j][1])
        pairs=len(labs)//2
        if pairs>=min_pairs:
            dx= max(xs)-min(xs); dy= max(ys)-min(ys)
            if min(dx,dy) <= max_depth:
                if dx >= dy:  # ratakan horizontal
                    y0=float(np.median(ys)); out.append(np.array([pts[j][0], y0])); i=j; continue
                else:         # ratakan vertikal
                    x0=float(np.median(xs)); out.append(np.array([x0, pts[j][1]])); i=j; continue
        out.append(pts[i+1]); i+=1
    out=np.asarray(out)
    if (out[0]!=out[-1]).any(): out=np.vstack([out, out[0]])
    return _remove_colinear_ring(out, COLLINEAR_TOL_DEG)

# C. hapus spike/notch berdasarkan kedalaman & sudut
def _remove_spikes_by_depth(coords, angle_max=70.0, depth_max=0.75):
    P = np.asarray(coords, float).copy()
    changed=True
    while changed:
        changed=False
        for i in range(1, len(P)-1):
            a, b, c = P[i-1], P[i], P[i+1]
            v1 = a-b; v2 = c-b
            L1 = float(np.hypot(*v1)); L2 = float(np.hypot(*v2))
            if L1<1e-9 or L2<1e-9: continue
            ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
            ang = min(ang, 360-ang)
            # kedalaman spike = jarak titik b ke garis (a-c)
            A=a; C=c
            AC=C-A; LAC=float(np.hypot(*AC))
            if LAC<1e-9: continue
            depth = abs(np.cross(AC, b-A))/LAC
            if ang < angle_max and depth <= depth_max:
                # buang apex b
                P = np.vstack([P[:i], P[i+1:]])
                changed=True
                break
    if (P[0]!=P[-1]).any(): P=np.vstack([P, P[0]])
    return _remove_colinear_ring(P, COLLINEAR_TOL_DEG)

# D. ganti bevel pendek jadi pojok 90°
def _fix_short_bevels(coords, diag_max=2.0, right_tol=15.0, max_shift=1.8):
    def is_H(v): return abs(math.degrees(math.atan2(v[1], v[0]))) <= right_tol or \
                      abs(180-abs(math.degrees(math.atan2(v[1], v[0])))) <= right_tol
    def is_V(v): return abs(abs(math.degrees(math.atan2(v[1], v[0]))) - 90) <= right_tol

    P = np.asarray(coords, float)
    if len(P)<5: return P
    Q=[P[0]]; n=len(P)
    for i in range(1, n-2):
        a=P[i-1]; b=P[i]; c=P[i+1]; d=P[i+2]
        v_prev = b-a; v_diag = c-b; v_next = d-c
        Ld = float(np.hypot(*v_diag))
        if Ld<=diag_max:
            # pola: {H or V} -> diag (pendek) -> {V or H}
            if (is_H(v_prev) and is_V(v_next)) or (is_V(v_prev) and is_H(v_next)):
                # bentuk sudut 90° di perpotongan garis H & V
                # ambil nilai x dari garis vertikal, y dari garis horizontal
                if is_V(v_next): x_corner = c[0]
                else:            x_corner = b[0]
                if is_H(v_prev): y_corner = b[1]
                else:            y_corner = c[1]
                corner = np.array([x_corner, y_corner], float)
                # jaga pergeseran tak kebablasan
                if np.hypot(*(corner-b)) <= max_shift and np.hypot(*(corner-c)) <= max_shift:
                    Q.append(corner)
                    # lewati c (hapus diagonal)
                    continue
        Q.append(b)
    Q.extend([P[-2], P[-1]])
    Q=np.asarray(Q)
    if (Q[0]!=Q[-1]).any(): Q=np.vstack([Q, Q[0]])
    return _remove_colinear_ring(Q, COLLINEAR_TOL_DEG)

# ---------- PIPELINE RING LOKAL ----------
def _process_ring(arr):
    # Ensure input array has at least 4 coordinates before processing
    if len(arr) < 4:
        return np.array([])  # Return empty array for invalid rings

    labs = _line_runs_classify(arr, ANG_CLASSIFY_DEG, ANG_FORCE_DEG)
    arr  = _apply_runs_snap(arr, labs, MERGE_LINE_TOL, GRID_SNAP, CORNER_SHIFT_MAX)
    arr  = _flatten_teeth(arr, teeth_max=STAIR_SEG_MAX*0.6)
    arr  = _flatten_staircase(arr, seg_max=STAIR_SEG_MAX, min_pairs=STAIR_MIN_PAIRS, max_depth=STAIR_MAX_DEPTH)
    arr  = _remove_spikes_by_depth(arr, angle_max=SPIKE_ANGLE_MAX, depth_max=SPIKE_DEPTH_MAX)
    arr  = _fix_short_bevels(arr, diag_max=BEVEL_DIAG_MAX, right_tol=RIGHT_TOL_DEG, max_shift=CORNER_SHIFT_MAX)

    # Ensure output array has at least 4 coordinates and is a valid LinearRing after processing
    if len(arr) < 4:
        return np.array([])

    processed_ring = LinearRing(arr)
    if not processed_ring.is_valid:
        return np.array([]) # Return empty array if the ring is invalid

    return arr


def _make_ortho_targeted(poly, med_angle):
    q = poly
    if SMOOTH_TOL>0: q = q.buffer(SMOOTH_TOL, join_style=2).buffer(-SMOOTH_TOL, join_style=2)
    if CLOSE_TOL>0:  q = q.buffer(CLOSE_TOL,  join_style=2).buffer(-CLOSE_TOL,  join_style=2)
    q = make_valid(q).buffer(0)

    rot = shp_rotate(q, -med_angle, origin='centroid', use_radians=False)

    def proc(p):
        # EXTERIOR
        ext = np.asarray(p.exterior.coords)
        ext2 = _process_ring(ext)

        # If exterior is invalid after processing, return None
        if len(ext2) < 4:
            return None

        # HOLES
        holes=[]
        for r in p.interiors:
            arr  = np.asarray(r.coords)
            rr = _process_ring(arr)
            # Only add valid holes with at least 4 coordinates
            if len(rr) >= 4:
                 holes.append(rr)

        # Attempt to create a Polygon and check its validity
        try:
            # Use the processed exterior and valid holes
            processed_polygon = Polygon(ext2, holes)
            if not processed_polygon.is_valid or processed_polygon.is_empty:
                return None
            return processed_polygon.buffer(0) # Clean up potential self-intersections
        except Exception:
            return None


    if isinstance(rot, Polygon):
        out_rot = proc(rot)
    else:
        parts = [proc(pg) for pg in rot.geoms]
        parts = [pp for pp in parts if pp is not None and not pp.is_empty]
        out_rot = unary_union(parts).buffer(0) if parts else None

    if out_rot is None: return None # Return None if the geometry became invalid

    out = shp_rotate(out_rot, med_angle, origin='centroid', use_radians=False)
    return make_valid(out).buffer(0)

def _area_match(poly, target, tol=AREA_TOL):
    if not AREA_MATCH: return poly
    A0 = poly.area
    if A0 >= target*(1.0 - tol): return poly
    lo, hi = 0.0, max(0.05, (target - A0)/max(poly.length, 1e-9) * 2.0)
    def area_of(d): return poly.buffer(d, join_style=2, mitre_limit=5.0).area
    while area_of(hi) < target and hi < 5.0: hi *= 2.0
    for _ in range(30):
        mid = 0.5*(lo+hi); Am = area_of(mid)
        if Am < target: lo = mid
        else: hi = mid
    return poly.buffer(hi, join_style=2, mitre_limit=5.0)

# ---------- RUN ----------
gdf = gpd.read_file(IN_PATH, engine="pyogrio")
gdf, work_crs = to_work_crs(gdf)
gdf["geometry"] = gdf.geometry.apply(lambda g: make_valid(g).buffer(0))
gdf = gdf[~gdf.geometry.is_empty].copy()

def regularize_right_angle_v3(geom):
    if geom is None or geom.is_empty: return geom
    theta = dominant_angle_deg(geom)
    ortho = _make_ortho_targeted(geom, theta)
    if ortho is None or ortho.is_empty: return geom
    return _area_match(ortho, geom.area, AREA_TOL).buffer(0)

out_geoms=[]
for g in tqdm(gdf.geometry, desc="Right-angle v3 targeted cleanup"):
    out_geoms.append(regularize_right_angle_v3(g))

gdf_out = gdf.copy()
gdf_out["geometry"] = out_geoms
gdf_out = gdf_out[~gdf_out.geometry.is_empty].copy()
gdf_out.to_file(OUT_PATH, driver="GeoJSON", engine="pyogrio")
print("Selesai →", OUT_PATH)

Right-angle v3 targeted cleanup:   0%|          | 0/54 [00:00<?, ?it/s]

/tmp/ipython-input-696780255.py:81: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:193: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:199: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  depth = abs(np.cross(AC, b-A))/LAC
/tmp/ipython-input-696780255.py:81: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:193: Deprecat

Selesai → /content/drive/MyDrive/Segmented_LAS/DATA TESIS/building_2x_RF/GSP_outline_FINISH.geojson


/tmp/ipython-input-696780255.py:81: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:81: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:81: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipython-input-696780255.py:193: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  ang = abs(math.degrees(math.atan2(np.cross(v1, v2), np.dot(v1, v2))))
/tmp/ipytho